In [ ]:
import os
import pandas as pd

## Download Yelp Dataset


Download the [Yelp Dataset](https://www.yelp.com/dataset), untar the data and store it under `./data` directory.


In [ ]:
data_dir = "./data"

businesses_fname = "yelp_academic_dataset_business.json"
reviews_fname = "yelp_academic_dataset_review.json"

## Yelp Businesses


In [ ]:
businesses = pd.read_json(os.path.join(data_dir, businesses_fname), lines=True)
businesses = businesses[["business_id", "stars", "review_count", "categories"]]
businesses.head()

In [ ]:
businesses.shape

In [ ]:
businesses.isnull().sum()

In [ ]:
businesses.dropna(inplace=True)
businesses.shape

In [ ]:
businesses[["stars"]].describe()

In [ ]:
businesses[["review_count"]].describe()

## Preprocess Yelp Businesses


Reviews from businesses with low ratings or low review counts may include noise, outliers, or uninformative content. So, we will filter out such businesses to ensure high data quality.


In [ ]:
min_review_count = 15
min_stars = 3.0

filtered_businesses = businesses[
    (businesses["review_count"] >= min_review_count)
    & (businesses["stars"] >= min_stars)
]

In [ ]:
filtered_businesses.shape

In [ ]:
filtered_businesses[["stars"]].describe()

In [ ]:
filtered_businesses[["review_count"]].describe()

## Yelp Reviews


In [ ]:
reviews = pd.read_json(os.path.join(data_dir, reviews_fname), lines=True)
reviews = reviews[["review_id", "business_id", "stars", "text"]]
reviews.head()

In [ ]:
reviews.shape

In [ ]:
reviews.isnull().sum()

In [ ]:
reviews[["stars"]].describe()

In [ ]:
# Review text length stats
reviews["text"].apply(len).describe()

In [ ]:
# Review # words stats
reviews["text"].apply(lambda x: len(x.split())).describe()

## Preprocess Reviews


Now filter out excessively long or short reviews, including ones with low ratings.


In [ ]:
min_stars = 3.0
filtered_reviews = reviews[
    (reviews["stars"] >= min_stars)
    & (
        reviews["text"].apply(
            lambda x: (len(x.split()) >= 42 and len(x.split()) <= 149)
        )
    )
]

In [ ]:
filtered_reviews.shape

In [ ]:
filtered_reviews[["stars"]].describe()

In [ ]:
# Review # words stats
filtered_reviews["text"].apply(lambda x: len(x.split())).describe()

## Group Businesses & Reviews


In [ ]:
# Now merge and filter out reviews from businesses that are not part of our list.
merged_df = pd.merge(
    filtered_reviews, filtered_businesses, on="business_id", how="inner"
)
merged_df.drop(columns=["review_count", "categories"], inplace=True)
merged_df.rename(
    columns={"stars_x": "review_stars", "stars_y": "business_stars"},
    inplace=True,
)
merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
merged_df["business_id"].nunique()

In [ ]:
merged_df.groupby(["business_id"])["business_id"].count().reset_index(
    name="count"
).describe()

In [ ]:
# Now group by business_id and filter out businesses that less # of reviews.
# Need to do this again because we already filtered a lot of reviews,
# so review_count for each businesses might be different now.
min_review_count = 15

yelp_dataset = merged_df.groupby(["business_id"]).filter(
    lambda x: len(x) >= min_review_count
)
yelp_dataset.head()

In [ ]:
yelp_dataset.shape

In [ ]:
yelp_dataset.groupby(["business_id"])["business_id"].count().reset_index(
    name="count"
).describe()

In [ ]:
yelp_dataset[["business_stars"]].describe()

In [ ]:
yelp_dataset[["review_stars"]].describe()

In [ ]:
# Review # words stats
yelp_dataset["text"].apply(lambda x: len(x.split())).describe()